<a href="https://colab.research.google.com/github/muradialotabi/DDoS-ATTACKS-IN-VANET-/blob/main/mycodecolabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')








Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# AE wit RF
import pandas as pd
import os
from matplotlib import pyplot
from numpy import where
import pandas as pd
import os
import json
import numpy as np
import pylab as plt
import time
import zipfile
import seaborn as sns
import random
from imblearn.over_sampling import SMOTE
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
import pandas as pd
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None  # default='warn'
import warnings
import pandas as pd
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


#def unzipfile(filename):
    #with zipfile.ZipFile(filename, "r") as zip_ref:
       #zip_ref.extractall()


# Unzip the files
# We assume the dataset file and the current python file are in the same folder.
# Otherwise, the path of the dataset file should be revised in the following line.
# unzipfile("VeReMi_57600_61200_2022-9-11_19.12.56.zip")

import os

path_1 = '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32'
path_2 = '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_28800_32400_2022-9-11_20_45_32'
path_3 = '/content/drive/MyDrive/Dataset/ConstPos_1416/VeReMi_50400_54000_2022-9-11_18_8_38'
path_4 = '/content/drive/MyDrive/Dataset/ConstPos_1416/VeReMi_54000_57600_2022-9-11_18_8_38'

# Calculate the number of json
list_of_files = []


for root, dirs, files in os.walk(path_1):

    for file in files:
        list_of_files.append(os.path.join(root, file))

for root, dirs, files in os.walk(path_2):
    for file in files:
       list_of_files.append(os.path.join(root, file))
for root, dirs, files in os.walk(path_3):
    for file in files:
       list_of_files.append(os.path.join(root, file))
for root, dirs, files in os.walk(path_4):
    for file in files:
       list_of_files.append(os.path.join(root, file))
#print(list_of_files)
list_of_files
# Each folder has one json file for the ground truth, we want to skip them
list_of_files.remove('/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceGroundTruthJSON-7.json')
list_of_files.remove('/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_28800_32400_2022-9-11_20_45_32/traceGroundTruthJSON-8.json')
list_of_files.remove('/content/drive/MyDrive/Dataset/ConstPos_1416/VeReMi_50400_54000_2022-9-11_18_8_38/traceGroundTruthJSON-14.json')
list_of_files.remove('/content/drive/MyDrive/Dataset/ConstPos_1416/VeReMi_54000_57600_2022-9-11_18_8_38/traceGroundTruthJSON-15.json')
# Calculate the number of json
nb_each_json = []

for index in range(len(list_of_files)):
    nb_each_json.append(sum(1 for line in open(list_of_files[index])))

len(list_of_files)
data = pd.DataFrame()
for index in range(len(list_of_files)):
    with open(list_of_files[index]) as f:
        # Read all the data in the files
        df = pd.DataFrame(json.loads(line) for line in f)
        # Read the following features from the name of json file (by splitting the file name)
        # labelRec: the label (0 normal, 13 (for this type of attack) attacker)
        # receiver: the receiver ID
        # moduleid: OMNeT++ module number, we don't need it now

        df['labelRec'] = list_of_files[index].rsplit('/')[-1].rsplit('-')[3].rsplit('A')[1]
        df['receiver'] = list_of_files[index].rsplit('/')[-1].rsplit('-')[1]
      # df['moduleid'] = list_of_files[index].rsplit('/')[-1].rsplit('-')[4]
        data = pd.concat([data, df])
    print(index)

data.head()
# Choose the BSM where type = 3 (type = 2 GPS is just the sent messages, we will use the received messages, i.e. BSM)

bsm = data[data.type == 3]


# Pos, Spd, Acl, Hed, and their noise features have three axis: x,y,z.
# This function separates their x and y axis (z is always 0 for all, so we did not consider them.)
def clean_dataset(l, droped_features, data):
    for t in l:
        data[t + '_x'] = None
        data[t + '_y'] = None

    for j in l:
        data[j + '_x'] = data[j].apply(lambda row: row[0])
        data[j + '_y'] = data[j].apply(lambda row: row[1])
    data = data.drop(columns=l, axis=1)
    data.drop_duplicates(inplace=True)
    data.drop(columns=droped_features, inplace=True)
    return data


# The features who needs to be separated for the different axis
l = ['pos', 'pos_noise', 'spd', 'spd_noise', 'acl', 'acl_noise', 'hed', 'hed_noise']
# Using clean_dataset function, separate the features in l to their x and y axis
droped_features = []
bsm = clean_dataset(l, droped_features, bsm)
bsm.head()
bsm.dtypes
bsm['receiver'] = bsm['receiver'].astype("float64")
# bsm['sender'] = bsm['sender'].astype("float64")
bsm['labelRec'] = bsm['labelRec'].astype("int64")

bsm.head()
# Receiver - Sender matching

labelization = bsm[['receiver', 'labelRec']]
labelization.drop_duplicates(inplace=True)
bsm = bsm.drop(columns=['receiver', 'labelRec'], axis=1)
bsm = pd.merge(bsm, labelization, left_on='sender', right_on='receiver', how='left')
bsm.rename(columns={'labelRec': 'label'}, inplace=True)
bsm = bsm.drop(columns=['receiver'], axis=1)
bsm = bsm.dropna()
# Separate features and labels
X = bsm.drop(columns=['label'], axis=1)
y = bsm['label']
# Create a new DataFrame with balanced data
bsm = pd.DataFrame(X, columns=X.columns)
bsm['label'] = y

#print(bsm)
# or:  bsm.dropna(inplace = True)
bsm.head()
bsm.describe()

bsm['label'] = bsm['label'].astype("int64")
# Number of lines in the dataset
print(len(bsm))


from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
def autoencoder(X_train, X_test):
    # Normalize the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the Autoencoder architecture
    input_dim = X_train.shape[1]
    encoding_dim = 32  # Adjust the encoding dimension as needed

    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation='relu')(input_layer)
    decoder = Dense(input_dim, activation='sigmoid')(encoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    #the decoder is implicitly defined as the
    # layer that follows the encoding layer in the autoencoder model. There is no separate decoder model defined.
#In this line, a dense layer is defined with input_dim neurons and a sigmoid activation function.
    # This layer serves as the decoder in the autoencoder model#
    # Compile and train the Autoencoder
    autoencoder.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
    autoencoder.fit(X_train_scaled, X_train_scaled, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test_scaled, X_test_scaled))
    # Use the trained Autoencoder to encode the data
    encoder = Model(inputs=input_layer, outputs=encoder)
    X_train_encoded = encoder.predict(X_train_scaled)
    X_test_encoded = encoder.predict(X_test_scaled)
    return X_train_encoded, X_test_encoded

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
def classification_attack(method, veremi):
    # Our labels are 0 or 13; we need 0-1 for classification algorithms. Convert 13 to 1
    veremi['label'] = veremi['label'].replace(13, 1)

    # Create feature set X and label set y
    y = veremi['label']
    X = veremi.drop(columns=['label', 'type', 'rcvTime', 'sendTime', 'sender', 'senderPseudo', 'messageID'], axis=1)

    # Data split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    if method == 'Autoencoder':

        # Autoencoder
        X_train_encoded, X_test_encoded = autoencoder(X_train, X_test)

        # Choose a classification algorithm (e.g., RandomForestClassifier) and fit it on the encoded data
        clf = RandomForestClassifier(n_estimators=50, bootstrap=True, random_state=0)

        # Measure the time taken for model training
        start_time = time.time()
        clf.fit(X_train_encoded, y_train)
        end_time = time.time()
        timefit = end_time - start_time

        # Predict on the encoded test data
        start_time = time.time()
        y_pred = clf.predict(X_test_encoded)
        end_time = time.time()
        timepred = end_time - start_time

        report = classification_report(y_test, y_pred, output_dict=True)
        return (report['accuracy'], report['weighted avg']['precision'], report['weighted avg']['recall'],
                report['weighted avg']['f1-score'], timefit, timepred)

results_classifying = pd.DataFrame(index=['Autoencoder'],
columns=['Acc', 'Pre', 'Rec', 'F1s', 'TimeFit', 'TimePred'])

results_classifying.xs('Autoencoder')[:] = classification_attack('Autoencoder', bsm)
print(results_classifying)

results_classifying.to_excel('results_classifying.xlsx')
# Storage of the data
bsm.to_csv('bsm.csv')
### Read the saved data
bsm_copy = pd.read_csv('bsm.csv')
bsm_copy.drop(columns='Unnamed: 0', inplace=True)
bsm_copy.head()
## Read the saved results
results_classifying_copy = pd.read_excel('results_classifying.xlsx')
results_classifying_copy.rename(columns={'Unnamed: 0': 'method'}, inplace=True)
results_classifying_copy.set_index('method', inplace=True)
print(results_classifying_copy)



['/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-5409-5407-A0-27070-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-5235-5233-A1-27034-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-4935-4933-A1-26944-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-5193-5191-A1-27025-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-4989-4987-A0-26969-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-5055-5053-A1-26982-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-5355-5353-A1-27062-7.json', '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32/traceJSON-5481-5479-A0-27083-7.json', '/content/drive/MyDrive

In [ ]:
import os

path_1 = '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_25200_28800_2022-9-11_20_45_32'
#unzipfile("VeReMi_28800_32400_2022-9-11_20.45.32.zip")
print(path_1)
path_2 = '/content/drive/MyDrive/Dataset/ConstPos_0709/VeReMi_28800_32400_2022-9-11_20_45_32'
path_3 = '/content/drive/MyDrive/Dataset/ConstPos_1416/VeReMi_50400_54000_2022-9-11_18_8_38'
path_4 = '/content/drive/MyDrive/Dataset/ConstPos_1416/VeReMi_54000_57600_2022-9-11_18_8_38'
#ist_of_files.remove('VeReMi_28800_32400_2022-9-11_20_45_32\\traceGroundTruthJSON-8.json')
# list_of_files.remove('VeReMi_54000_57600_2022-9-11_19_12_56\\traceGroundTruthJSON-15.json')
# Calculate the number of json
list_of_files = []


for root, dirs, files in os.walk(path_1):

    for file in files:
        list_of_files.append(os.path.join(root, file))
        print(list_of_files)
for root, dirs, files in os.walk(path_2):
    for file in files:
        list_of_files.append(os.path.join(root, file))


print(list_of_files)